In [30]:
import os
import json
import datetime
import pytz
import mimetypes
import csv
from io import StringIO
from google.cloud import storage


def get_kr_time():
    korea_timezone = pytz.timezone('Asia/Seoul')
    current_time_korea = datetime.now(pytz.utc).astimezone(korea_timezone)
    return current_time_korea.strftime('%Y-%m-%d %H:%M:%S')


# Setting

In [4]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "./key.json"

client = storage.Client()
bucket_name = 'contact-419811.appspot.com' 
bucket = client.bucket(bucket_name)

# Save data to google cloud

In [31]:
def upload_file_to_gcs(bucket, filename, data, is_binary=False):
    blob = bucket.blob(filename)
    content_type, _ = mimetypes.guess_type(filename)
    
    if is_binary:
        blob.upload_from_string(data, content_type=content_type)
    else:
        if content_type == 'application/json' and isinstance(data, dict):
            data = json.dumps(data)
        elif content_type == 'text/csv' and isinstance(data, list):
            output = StringIO()
            writer = csv.DictWriter(output, fieldnames=data[0].keys())
            writer.writeheader()
            writer.writerows(data)
            data = output.getvalue()
        
        blob.upload_from_string(data, content_type=content_type)

    print(f"Uploaded {filename} to {bucket.name}")

In [2]:


def get_data(dir):
    bucket = client.get_bucket(bucket_name)
    blobs = bucket.list_blobs(prefix = dir)
    all_data = []       
    for blob in blobs:
        # Assuming your data is stored in JSON format
        blob_content = blob.download_as_text()
        data = json.loads(blob_content)
        all_data.append(data)
        
    all_data = sorted(all_data, key=lambda x: datetime.strptime(x['time'], '%Y-%m-%d %H:%M:%S'), reverse = True)

    return all_data

In [34]:
def list_files_in_bucket(bucket):
    blobs = bucket.list_blobs()
    file_list = [blob.name for blob in blobs]
    return file_list

# Example usage:
file_list = list_files_in_bucket(bucket)
print("Files in bucket:")
for file in file_list:
    print(file)

Files in bucket:
data/1232024-04-12 16:19:07.json
data/Hyeok Jong Lee2024-04-10 18:22:10.json
data/JEYOON YEOM2024-04-10 21:58:26.json
data/Junbo Jang2024-04-10 21:58:13.json
data/Lee Hyeokjong2024-04-12 16:43:12.json
data/ad2024-04-10 18:07:13.json
data/asd2024-04-10 22:43:42.json
data/asdf2024-04-10 18:09:18.json
data/asdf2024-05-09 22:12:43.json
data/test2024-04-10 17:56:53.json
data/권소영2024-04-18 22:09:46.json
data/김경준2024-04-18 22:09:50.json
data/김도현2024-04-11 13:04:19.json
data/김유진2024-05-09 22:08:45.json
data/김재원2024-04-17 18:41:02.json
data/박상우2024-04-19 15:27:29.json
data/안준현2024-05-23 16:24:09.json
data/오지안2024-06-07 17:16:30.json
data/이설하2024-04-19 18:07:56.json
data/이혁종 테스트2024-04-10 22:30:41.json
data/이혁종2024-04-10 17:28:52.json
data/이혁종2024-04-10 18:30:33.json
data/이혁종2024-04-12 16:56:05.json
data/재여니2024-04-11 19:15:26.json
data/재연2024-04-11 19:19:32.json
data/테스트2024-04-10 22:31:42.json
data/테스트해보기2024-04-11 19:18:40.json
login_logs/2024-04-10 17:29:07.json
login_logs/2